# KB2 Construction -  Fast-Retrieval Knowledge Base

In [11]:
# Cell 1: Config & Dependencies
import json
import numpy as np
import faiss
from pathlib import Path
from collections import Counter, defaultdict
from datetime import datetime
import pickle
import gzip
import gc
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

# Project paths
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name=='notebooks' else Path.cwd()
CPG_JSON_DIR = PROJECT_ROOT/'data'/'tmp'/'cpg_json'
OUTPUT_DIR = PROJECT_ROOT/'data'/'processed'
OUTPUT_DIR.mkdir(exist_ok=True)

In [12]:
# Cell 2: Load KB1 Metadata & Utils

# Path to the KB1 metadata file
kb1_path = PROJECT_ROOT/'data'/'processed'/'kb1.json'
kb1 = json.load(open(kb1_path, encoding='utf-8'))

def build_kb1_index(data):
    """
    Build KB1 index using existing composite keys.
    Returns:
        - kb1_by_composite: dict mapping composite_key to entry data
        - kb1_by_cve: dict mapping CVE ID to list of entries
    """
    kb1_by_composite = {}
    kb1_by_cve = defaultdict(list)
    
    for composite_key, entry_data in data.items():
        # Use the existing composite keys from KB1
        kb1_by_composite[composite_key] = entry_data
        
        # Index by CVE for lookup
        cve_id = entry_data.get('cve_id', '')
        if cve_id:
            kb1_by_cve[cve_id].append({
                'composite_key': composite_key,
                'cwe_id': entry_data.get('cwe_id', ''),
                'data': entry_data
            })
    
    return kb1_by_composite, kb1_by_cve

# Build the KB1 indices
kb1_by_composite, kb1_by_cve = build_kb1_index(kb1)

def extract_cve_from_path(p): 
    """
    Extract CVE ID from a file path.
    Returns the first path segment starting with 'CVE-'.
    """
    return next((pt.split('_')[0] for pt in str(p).split('/') if pt.startswith('CVE-')), None)

def get_meta_from_kb1(composite_key, kb1_data):
    """
    Get metadata from KB1 using an existing composite key.
    Returns the KB1 entry if found, else None.
    """
    if composite_key in kb1_data:
        # Directly return the KB1 entry
        return kb1_data[composite_key]
    return None

# Test KB1 integration: print summary statistics
print(f"📊 KB1 loaded: {len(kb1_by_composite)} entries")
print(f"📊 Unique CVEs in KB1: {len(kb1_by_cve)}")

# Show a sample KB1 entry structure
if kb1_by_composite:
    sample_key = list(kb1_by_composite.keys())[0]
    sample_entry = kb1_by_composite[sample_key]
    print(f"📄 Sample KB1 entry: {sample_key}")
    print(f"   • CVE: {sample_entry.get('cve_id', 'N/A')}")
    print(f"   • CWE: {sample_entry.get('cwe_id', 'N/A')}")
    print(f"   • Description: {sample_entry.get('vulnerability_type', 'N/A')[:50]}...")

📊 KB1 loaded: 2317 entries
📊 Unique CVEs in KB1: 1154
📄 Sample KB1 entry: CWE-119_CVE-2014-3182_0
   • CVE: CVE-2014-3182
   • CWE: CWE-119
   • Description: Invalid user input provided to the device index, w...


In [13]:
# Cell 3: Find Vulnerability Pairs
def find_pairs(dir_):
    """
    Find all vulnerability/patch pairs in the CPG directory using the correct composite keys.
    This function iterates over CWE directories and their instances, looking for both
    'vuln_cpg.json' and 'patch_cpg.json' files. If both files exist, it constructs a composite key
    (matching the KB1 format) and stores relevant metadata for each pair.
    """
    pairs = {}
    for cwe_dir in dir_.iterdir():
        # Only process directories whose names start with 'CWE-'
        if not cwe_dir.name.startswith('CWE-'):
            continue
        for inst in cwe_dir.iterdir():
            vuln_file = inst / 'vuln_cpg.json'
            patch_file = inst / 'patch_cpg.json'
            # Check that both the vulnerable and patch CPG files exist
            if vuln_file.exists() and patch_file.exists():
                cve = extract_cve_from_path(vuln_file)
                # Extract the instance number from the directory name
                num = inst.name.split('_')[-1]
                # Construct the composite key in the same format as KB1
                composite_key = f"{cwe_dir.name}_{cve}_{num}"
                pairs[composite_key] = {
                    'vuln': vuln_file,
                    'safe': patch_file,
                    'cve': cve,
                    'cwe': cwe_dir.name,
                    'instance_id': num,
                    'composite_key': composite_key  # Composite key for cross-referencing
                }
    return pairs

CPG_DIR = PROJECT_ROOT / 'data' / 'tmp' / 'cpg_json'
pairs = find_pairs(CPG_DIR)
print(f"Found {len(pairs)} vulnerability/patch pairs")

Found 2317 vulnerability/patch pairs


In [14]:
# Cell 4: Import Feature Extraction
import sys
sys.path.append(str(PROJECT_ROOT/'scripts'))
from kb2_preprocessing import extract_kb2_features

# Test feature extraction
sample_pair = next(iter(pairs.items()))
sample_key, sample_info = sample_pair
vf = extract_kb2_features(sample_info['vuln'])
sf = extract_kb2_features(sample_info['safe'])

print(f"Sample extraction - Quality: {vf['extraction_metadata']['feature_completeness']['quality_score']:.2f}")
print(f"Structural features: {len(vf['structural_features'])} dimensions")
print(f"Semantic features: {len(vf['semantic_features'])} elements")

✅ Loaded CWE configuration from /Users/vernetemmanueladjobi/Documents/RessourcesStages/Projets/VulRAG-Hybrid-System/scripts/kb2_preprocessing/config/cwe_config.json
   - 9 CWE patterns loaded
   - 9 risk weights configured
✅ Loaded CWE configuration from /Users/vernetemmanueladjobi/Documents/RessourcesStages/Projets/VulRAG-Hybrid-System/scripts/kb2_preprocessing/config/cwe_config.json
   - 9 CWE patterns loaded
   - 9 risk weights configured
Sample extraction - Quality: 1.00
Structural features: 12 dimensions
Semantic features: 7 elements


In [15]:
def load_code_body(composite_key, kb1_meta):
    """Load code body directly from temp_code_files"""
    # Parse composite key: CWE-416_CVE-2014-3182_1
    parts = composite_key.split('_')
    if len(parts) < 3:
        return None, None
    
    cwe = parts[0]  # CWE-416
    cve = parts[1]  # CVE-2014-3182
    instance = parts[2]  # 1
    
    # Build file paths with correct format
    temp_dir = PROJECT_ROOT/'data'/'tmp'/'temp_code_files'
    vuln_file = temp_dir/cwe/f"{cve}_{instance}_vuln.c"
    patch_file = temp_dir/cwe/f"{cve}_{instance}_patch.c"
    
    # Load code
    vuln_code = vuln_file.read_text() if vuln_file.exists() else None
    patch_code = patch_file.read_text() if patch_file.exists() else None
    
    return vuln_code, patch_code

sample_key = list(pairs.keys())[0]
sample_info = pairs[sample_key]
sample_kb1 = get_meta_from_kb1(sample_key, kb1_by_composite)
vuln_code, patch_code = load_code_body(sample_key, sample_kb1)

print(f"Sample VULN code length: {len(vuln_code) if vuln_code else 0} chars")
print(f"Sample PATCH code length: {len(patch_code) if patch_code else 0} chars")

Sample VULN code length: 445 chars
Sample PATCH code length: 395 chars


In [16]:
# Cell 6: OpenAI Embeddings Setup
from openai import OpenAI
import os

# Initialize OpenAI client
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    print("⚠️  OPENAI_API_KEY not found. Embeddings will be skipped.")
    client = None
else:
    client = OpenAI(api_key=api_key)
    print("✅ OpenAI client initialized")

def generate_embedding(code):
    """Generate OpenAI embedding for code snippet"""
    if not client or not code:
        return None
    
    try:
        response = client.embeddings.create(
            model="text-embedding-ada-002",
            input=code
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"Error generating embedding: {e}")
        return None

✅ OpenAI client initialized


In [17]:
# Cell 7: Build KB2 with Vector Separation - DOUBLE INDEXATION
def build_kb2_(pairs, limit=None):
    """
    Build KB2 with DOUBLE INDEXATION: VULN vs PATCH.
    This function constructs the KB2 entries, generating separate embeddings, structural features,
    and TF-IDF texts for both the vulnerable and patched code, and associates each entry with its composite key.
    """
    kb2_entries = []

    # Double embeddings: separate lists for vulnerable and patched code
    vuln_embeddings = []
    patch_embeddings = []

    # Double structural features: separate lists for vulnerable and patched code
    vuln_structural_features = []
    patch_structural_features = []

    # Double TF-IDF: separate lists for vulnerable and patched code
    vuln_tfidf_texts = []
    patch_tfidf_texts = []

    # Limit the number of items if a limit is specified
    items = list(pairs.items())[:limit] if limit else pairs.items()

    print(f"Building KB2 with DOUBLE INDEXATION for {len(items)} pairs...")

    for embed_key, (composite_key, info) in enumerate(tqdm(items)):
        # Extract features for the vulnerable code
        vf = extract_kb2_features(info['vuln'])
        if 'error' in vf:
            print(f"❌ Failed to extract vulnerable features for {composite_key}")
            continue

        # Extract features for the patched code
        sf = extract_kb2_features(info['safe'])
        if 'error' in sf:
            print(f"❌ Failed to extract patch features for {composite_key}")
            continue

        # Retrieve metadata from KB1 using the composite key
        kb1_meta = get_meta_from_kb1(composite_key, kb1_by_composite)

        # Load code bodies for embedding generation
        vuln_code, patch_code = load_code_body(composite_key, kb1_meta)

        # Generate embeddings for both vulnerable and patched code
        vuln_embedding = generate_embedding(vuln_code) if vuln_code else None
        patch_embedding = generate_embedding(patch_code) if patch_code else None

        # Construct the KB2 entry with all relevant metadata and features
        entry = {
            'composite_key': composite_key,  # Composite key from KB1
            'cve': info['cve'],
            'cwe': info['cwe'],
            'instance_id': info['instance_id'],
            'kb1_metadata': kb1_meta,  # Complete KB1 metadata

            # Separate features for VULN and PATCH
            'vuln_features': {
                'structural_features': vf['structural_features'],
                'semantic_features': vf['semantic_features'],
                'code_before_change': kb1_meta['code_before_change'] if kb1_meta else ''
            },
            'patch_features': {
                'structural_features': sf['structural_features'],
                'semantic_features': sf['semantic_features'],
                'code_after_change': kb1_meta['code_after_change'] if kb1_meta else ''
            },

            # Separate embedding keys for VULN and PATCH
            'vuln_embed_key': len(vuln_embeddings) if vuln_embedding else None,
            'patch_embed_key': len(patch_embeddings) if patch_embedding else None,

            # Timestamp for entry creation
            'ts': datetime.now().isoformat()
        }

        # Store the vulnerable code's embedding, structural features, and TF-IDF text if available
        if vuln_embedding:
            vuln_embeddings.append(vuln_embedding)
            vuln_structural_features.append(list(vf['structural_features'].values()))
            vuln_tfidf_texts.append(vf['semantic_features'].get('combined_text', ''))

        # Store the patched code's embedding, structural features, and TF-IDF text if available
        if patch_embedding:
            patch_embeddings.append(patch_embedding)
            patch_structural_features.append(list(sf['structural_features'].values()))
            patch_tfidf_texts.append(sf['semantic_features'].get('combined_text', ''))

        # Add the entry to the KB2 entries list
        kb2_entries.append(entry)

    # Return all constructed data structures for further processing or saving
    return kb2_entries, vuln_embeddings, patch_embeddings, vuln_structural_features, patch_structural_features, vuln_tfidf_texts, patch_tfidf_texts

In [18]:
# Cell 8: Build Full KB2 with Double Indexation
print("Starting full KB2 build with DOUBLE INDEXATION...")
kb2_entries, vuln_embeddings, patch_embeddings, vuln_structural_features, patch_structural_features, vuln_tfidf_texts, patch_tfidf_texts = build_kb2_(pairs)

print(f"✅ Built KB2 with {len(kb2_entries)} entries")
print(f"📊 VULN embeddings: {len(vuln_embeddings)} vectors")
print(f"📊 PATCH embeddings: {len(patch_embeddings)} vectors")
print(f"📊 VULN structural features: {len(vuln_structural_features)} vectors")
print(f"📊 PATCH structural features: {len(patch_structural_features)} vectors")
print(f"📊 VULN TF-IDF texts: {len(vuln_tfidf_texts)} texts")
print(f"📊 PATCH TF-IDF texts: {len(patch_tfidf_texts)} texts")

Starting full KB2 build with DOUBLE INDEXATION...
Building KB2 with DOUBLE INDEXATION for 2317 pairs...


  0%|          | 0/2317 [00:00<?, ?it/s]

✅ Loaded CWE configuration from /Users/vernetemmanueladjobi/Documents/RessourcesStages/Projets/VulRAG-Hybrid-System/scripts/kb2_preprocessing/config/cwe_config.json
   - 9 CWE patterns loaded
   - 9 risk weights configured
✅ Loaded CWE configuration from /Users/vernetemmanueladjobi/Documents/RessourcesStages/Projets/VulRAG-Hybrid-System/scripts/kb2_preprocessing/config/cwe_config.json
   - 9 CWE patterns loaded
   - 9 risk weights configured


  0%|          | 1/2317 [00:01<41:20,  1.07s/it]

✅ Loaded CWE configuration from /Users/vernetemmanueladjobi/Documents/RessourcesStages/Projets/VulRAG-Hybrid-System/scripts/kb2_preprocessing/config/cwe_config.json
   - 9 CWE patterns loaded
   - 9 risk weights configured
✅ Loaded CWE configuration from /Users/vernetemmanueladjobi/Documents/RessourcesStages/Projets/VulRAG-Hybrid-System/scripts/kb2_preprocessing/config/cwe_config.json
   - 9 CWE patterns loaded
   - 9 risk weights configured


...

✅ Loaded CWE configuration from /Users/vernetemmanueladjobi/Documents/RessourcesStages/Projets/VulRAG-Hybrid-System/scripts/kb2_preprocessing/config/cwe_config.json
   - 9 CWE patterns loaded
   - 9 risk weights configured
✅ Loaded CWE configuration from /Users/vernetemmanueladjobi/Documents/RessourcesStages/Projets/VulRAG-Hybrid-System/scripts/kb2_preprocessing/config/cwe_config.json
   - 9 CWE patterns loaded
   - 9 risk weights configured


100%|█████████▉| 2316/2317 [35:00<00:00,  1.33it/s]

✅ Loaded CWE configuration from /Users/vernetemmanueladjobi/Documents/RessourcesStages/Projets/VulRAG-Hybrid-System/scripts/kb2_preprocessing/config/cwe_config.json
   - 9 CWE patterns loaded
   - 9 risk weights configured
✅ Loaded CWE configuration from /Users/vernetemmanueladjobi/Documents/RessourcesStages/Projets/VulRAG-Hybrid-System/scripts/kb2_preprocessing/config/cwe_config.json
   - 9 CWE patterns loaded
   - 9 risk weights configured


100%|██████████| 2317/2317 [35:02<00:00,  1.10it/s]

✅ Built KB2 with 2317 entries
📊 VULN embeddings: 2313 vectors
📊 PATCH embeddings: 2313 vectors
📊 VULN structural features: 2313 vectors
📊 PATCH structural features: 2313 vectors
📊 VULN TF-IDF texts: 2313 texts
📊 PATCH TF-IDF texts: 2313 texts


In [19]:
# Cell 9: Save Metadata (kb2.json) - DOUBLE INDEXATION
# ✅ Uses the composite keys from KB1 with separate embed_keys
kb2_dict = {entry['composite_key']: entry for entry in kb2_entries}

with open(OUTPUT_DIR/'kb2.json', 'w') as f:
    json.dump(kb2_dict, f, indent=2)

print(f"✅ Saved kb2.json with {len(kb2_dict)} entries")
print(f"📊 Unique CVEs: {len(set(entry['cve'] for entry in kb2_entries))}")
print(f"📊 Unique CWEs: {len(set(entry['cwe'] for entry in kb2_entries))}")

✅ Saved kb2.json with 2317 entries
📊 Unique CVEs: 1154
📊 Unique CWEs: 10


In [20]:
# Cell 10: Save Double Embeddings (VULN + PATCH)
print("\n🔍 Saving DOUBLE EMBEDDINGS...")

# ✅ VULN Embeddings
if vuln_embeddings:
    vuln_embeddings_array = np.array(vuln_embeddings, dtype='float32')
    np.save(OUTPUT_DIR/'kb2_vuln_embeddings.npy', vuln_embeddings_array)
    print(f"✅ Saved kb2_vuln_embeddings.npy: {vuln_embeddings_array.shape}")
    
    # Create FAISS index for VULN
    faiss.normalize_L2(vuln_embeddings_array)
    vuln_index = faiss.IndexFlatIP(vuln_embeddings_array.shape[1])
    vuln_index.add(vuln_embeddings_array)
    faiss.write_index(vuln_index, str(OUTPUT_DIR/'kb2_vuln_embeddings.faiss'))
    print(f"✅ Saved kb2_vuln_embeddings.faiss index")
else:
    print("⚠️  No VULN embeddings to save")

# ✅ PATCH Embeddings
if patch_embeddings:
    patch_embeddings_array = np.array(patch_embeddings, dtype='float32')
    np.save(OUTPUT_DIR/'kb2_patch_embeddings.npy', patch_embeddings_array)
    print(f"✅ Saved kb2_patch_embeddings.npy: {patch_embeddings_array.shape}")
    
    # Create FAISS index for PATCH
    faiss.normalize_L2(patch_embeddings_array)
    patch_index = faiss.IndexFlatIP(patch_embeddings_array.shape[1])
    patch_index.add(patch_embeddings_array)
    faiss.write_index(patch_index, str(OUTPUT_DIR/'kb2_patch_embeddings.faiss'))
    print(f"✅ Saved kb2_patch_embeddings.faiss index")
else:
    print("⚠️  No PATCH embeddings to save")


🔍 Saving DOUBLE EMBEDDINGS...
✅ Saved kb2_vuln_embeddings.npy: (2313, 1536)
✅ Saved kb2_vuln_embeddings.faiss index
✅ Saved kb2_patch_embeddings.npy: (2313, 1536)
✅ Saved kb2_patch_embeddings.faiss index


In [21]:
# Cell 11: Save Double Structural Features (VULN + PATCH)
print("\n🏗️ Saving DOUBLE STRUCTURAL FEATURES...")

# ✅ VULN Structural Features
if vuln_structural_features:
    vuln_scaler = StandardScaler()
    vuln_struct_array = np.array(vuln_structural_features, dtype='float32')
    vuln_struct_normalized = vuln_scaler.fit_transform(vuln_struct_array)
    
    np.save(OUTPUT_DIR/'kb2_vuln_struct.npy', vuln_struct_normalized)
    joblib.dump(vuln_scaler, OUTPUT_DIR/'kb2_vuln_struct_scaler.pkl')
    print(f"✅ Saved kb2_vuln_struct.npy: {vuln_struct_normalized.shape}")
    print(f"✅ Saved kb2_vuln_struct_scaler.pkl")
else:
    print("⚠️  No VULN structural features to save")

# ✅ PATCH Structural Features
if patch_structural_features:
    patch_scaler = StandardScaler()
    patch_struct_array = np.array(patch_structural_features, dtype='float32')
    patch_struct_normalized = patch_scaler.fit_transform(patch_struct_array)
    
    np.save(OUTPUT_DIR/'kb2_patch_struct.npy', patch_struct_normalized)
    joblib.dump(patch_scaler, OUTPUT_DIR/'kb2_patch_struct_scaler.pkl')
    print(f"✅ Saved kb2_patch_struct.npy: {patch_struct_normalized.shape}")
    print(f"✅ Saved kb2_patch_struct_scaler.pkl")
else:
    print("⚠️  No PATCH structural features to save")


🏗️ Saving DOUBLE STRUCTURAL FEATURES...
✅ Saved kb2_vuln_struct.npy: (2313, 12)
✅ Saved kb2_vuln_struct_scaler.pkl
✅ Saved kb2_patch_struct.npy: (2313, 12)
✅ Saved kb2_patch_struct_scaler.pkl


In [22]:
# Cell 12: Save Double TF-IDF (VULN + PATCH)
print("\n📝 Saving DOUBLE TF-IDF...")

# ✅ VULN TF-IDF
if vuln_tfidf_texts:
    vuln_tfidf_vectorizer = TfidfVectorizer(
        max_features=1000,
        min_df=2,
        max_df=0.95,
        ngram_range=(1, 2)
    )
    
    vuln_tfidf_matrix = vuln_tfidf_vectorizer.fit_transform(vuln_tfidf_texts)
    
    # Save VULN vectorizer and matrix
    joblib.dump(vuln_tfidf_vectorizer, OUTPUT_DIR/'kb2_vuln_tfidf.pkl')
    joblib.dump(vuln_tfidf_matrix, OUTPUT_DIR/'kb2_vuln_tfidf.npz')
    
    print(f"✅ Saved kb2_vuln_tfidf.pkl (vectorizer)")
    print(f"✅ Saved kb2_vuln_tfidf.npz: {vuln_tfidf_matrix.shape}")
else:
    print("⚠️  No VULN TF-IDF texts to save")

# ✅ PATCH TF-IDF
if patch_tfidf_texts:
    patch_tfidf_vectorizer = TfidfVectorizer(
        max_features=1000,
        min_df=2,
        max_df=0.95,
        ngram_range=(1, 2)
    )
    
    patch_tfidf_matrix = patch_tfidf_vectorizer.fit_transform(patch_tfidf_texts)
    
    # Save PATCH vectorizer and matrix
    joblib.dump(patch_tfidf_vectorizer, OUTPUT_DIR/'kb2_patch_tfidf.pkl')
    joblib.dump(patch_tfidf_matrix, OUTPUT_DIR/'kb2_patch_tfidf.npz')
    
    print(f"✅ Saved kb2_patch_tfidf.pkl (vectorizer)")
    print(f"✅ Saved kb2_patch_tfidf.npz: {patch_tfidf_matrix.shape}")
else:
    print("⚠️  No PATCH TF-IDF texts to save")


📝 Saving DOUBLE TF-IDF...
✅ Saved kb2_vuln_tfidf.pkl (vectorizer)
✅ Saved kb2_vuln_tfidf.npz: (2313, 1000)
✅ Saved kb2_patch_tfidf.pkl (vectorizer)
✅ Saved kb2_patch_tfidf.npz: (2313, 1000)


In [23]:
# Cell 13: Usage Example - DOUBLE INDEXATION
print("\n" + "="*50)
print("USAGE EXAMPLE - DOUBLE INDEXATION:")
print("="*50)

print("""
# Load KB2 for DOUBLE RETRIEVAL
import faiss, numpy as np, json

# Load metadata
with open('data/processed/kb2.json') as f:
    kb2 = json.load(f)

# Load VULN vectors
vuln_embeddings = np.load('data/processed/kb2_vuln_embeddings.npy', mmap_mode='r')
vuln_index = faiss.read_index('data/processed/kb2_vuln_embeddings.faiss')

# Load PATCH vectors  
patch_embeddings = np.load('data/processed/kb2_patch_embeddings.npy', mmap_mode='r')
patch_index = faiss.read_index('data/processed/kb2_patch_embeddings.faiss')

# Get vectors for entry
def get_vuln_vector(entry_id):
    entry = kb2[entry_id]
    if entry['vuln_embed_key'] is not None:
        return vuln_embeddings[entry['vuln_embed_key']]
    return None

def get_patch_vector(entry_id):
    entry = kb2[entry_id]
    if entry['patch_embed_key'] is not None:
        return patch_embeddings[entry['patch_embed_key']]
    return None

# Example usage
sample_entry = list(kb2.keys())[0]
vuln_vector = get_vuln_vector(sample_entry)
patch_vector = get_patch_vector(sample_entry)
print(f"VULN vector for {sample_entry}: shape {vuln_vector.shape if vuln_vector is not None else 'None'}")
print(f"PATCH vector for {sample_entry}: shape {patch_vector.shape if patch_vector is not None else 'None'}")

# ✅ Multi-CWE CVE handling with KB1 metadata
cve_entries = [entry for entry in kb2.values() if entry['cve'] == 'CVE-2014-3182']
print(f"Found {len(cve_entries)} entries for CVE-2014-3182:")
for entry in cve_entries:
    kb1_meta = entry['kb1_metadata']
    print(f"  • {entry['composite_key']} (CWE: {entry['cwe']})")
    print(f"    VULN embed_key: {entry['vuln_embed_key']}")
    print(f"    PATCH embed_key: {entry['patch_embed_key']}")
    print(f"    Description: {kb1_meta.get('description', 'N/A')[:50]}...")
    print(f"    Solution: {kb1_meta.get('solution', 'N/A')[:50]}...")
""")

print(f"\n✅ KB2 DOUBLE INDEXATION complete!")
print(f"📁 Output files in: {OUTPUT_DIR}")
print(f"📊 Total entries: {len(kb2_entries)}")
print(f"🔑 Composite keys from KB1: {len(set(entry['composite_key'] for entry in kb2_entries))}")
print(f"🎯 Ready for VULN vs PATCH hybrid retrieval!")


USAGE EXAMPLE - DOUBLE INDEXATION:

# Load KB2 for DOUBLE RETRIEVAL
import faiss, numpy as np, json

# Load metadata
with open('data/processed/kb2.json') as f:
    kb2 = json.load(f)

# Load VULN vectors
vuln_embeddings = np.load('data/processed/kb2_vuln_embeddings.npy', mmap_mode='r')
vuln_index = faiss.read_index('data/processed/kb2_vuln_embeddings.faiss')

# Load PATCH vectors  
patch_embeddings = np.load('data/processed/kb2_patch_embeddings.npy', mmap_mode='r')
patch_index = faiss.read_index('data/processed/kb2_patch_embeddings.faiss')

# Get vectors for entry
def get_vuln_vector(entry_id):
    entry = kb2[entry_id]
    if entry['vuln_embed_key'] is not None:
        return vuln_embeddings[entry['vuln_embed_key']]
    return None

def get_patch_vector(entry_id):
    entry = kb2[entry_id]
    if entry['patch_embed_key'] is not None:
        return patch_embeddings[entry['patch_embed_key']]
    return None

# Example usage
sample_entry = list(kb2.keys())[0]
vuln_vector = get_vuln_ve